In [36]:
import numpy as np
import scipy.linalg as la
from scipy.fftpack import fft
import matplotlib.pyplot as plt
%matplotlib inline
# try to load a local style sheet for matplotlib
# see http://matplotlib.org/users/style_sheets.html
try:
    plt.style.use('presentation')
except:
    pass # do nothing if it cannot be found

ModuleNotFoundError: No module named 'scipy'

In [42]:
def gauss(f, n):
    k = np.arange(1.0, n + 1.0)
    beta = 0.5 / np.sqrt(1 - (2*k)**(-2)) # 3-term recurrence coeffs
    T = np.zeros((2,n+1)) # set up Jacobi matrix with 0 diagonal
    T[1,:-1] = beta # and beta on secondary diagonals
    E, V = la.eig_banded(T, lower=True) # compute eigenvalues and eigenvectors
    perm = np.argsort(E) # sort eigenvalues
    x = E[perm] # get nodes (= Legendre points)
    w = 2 * V[0,perm]**2 # get quadrature weights
    return w.dot(f(x))

def clenshaw_curtis(f, n = 32, ofx = None):
    if ofx:
        assert(len(fx) == n / 2)
        ofx = ofx / 2 # halve old fx's to account for doubled n
        k = np.arange(1, n, 2) 
        nfx = np.cos(np.pi * k / n) # eval missing points
        
        # interleave them
        fx = np.zeros(n + 1)
        fx[0::2] = ofx
        fx[1::2] = nfx 
    else:    
        k = np.arange(n + 1.0)  
        x = np.cos(np.pi * k / n) # Chebyshev points
        fx = f(x) / (2*n) # evaluate integrand
    g = fft(np.concatenate((fx, fx[-2:0:-1]))).real # fast Fourier transform
    a = np.concatenate(([g[0]], g[1:n] + g[2*n-1:n:-1], [g[n]])) # Chebyshev coefficients
    w = np.zeros(a.shape[0]) # set up weights
    w[0::2] = 2.0 / (1-np.arange(0, n+1, 2)**2)
    
    if not abs(w[-16:]).dot(abs(a[-16:])) <= 1e-15 * abs(gauss(f, n)):
        if not all(abs(a) <= 1e-15):
            clenshaw_curtis(f, n * 2, fx)
    else:
        plt.figure(figsize=(10,8))
        plt.semilogy(range(1, n), abs(a), 'ro-')
        plt.semilogy(range(1, n), abs(w).dot(abs(a)), 'rb-')
        plt.show()
        return (w.dot(a), n)

In [44]:
f = lambda x: (1+x) * x**20

N = 26
list_of_n = range(1, N)
I_gauss = np.empty(N)
I_clencurt = np.empty(N)
for n in list_of_n:
    I_gauss[n] = gauss(f, n)
    I_clencurt[n] = clenshaw_curtis(f, n)
I = gauss(f, 2*N)

plt.figure(figsize=(10,8))
err_gauss = np.maximum(1e-16, abs(I_gauss[list_of_n] - I))
err_clencurt = np.maximum(1e-16, abs(I_clencurt[list_of_n] - I))
plt.semilogy(list_of_n, err_gauss, 'ro-')
plt.semilogy(list_of_n, err_clencurt, 'bd-')
plt.xlabel('$n$')
plt.ylabel('Quadraturfehler')
plt.legend(('Gauss', 'Clenshaw-Curtis'))
plt.show()

NameError: name 'la' is not defined